In [1]:
from datetime import datetime
import os
import time
import requests
import json

import pickle
from pathlib import Path
import traceback

In [2]:
# call API to retrieve comments of a game with a game id

# game_steamid = 1716740          # starfield
# game_name = 'starfield'

# game_steamid = 2138330          # cyberpunk2077 phantom liberty
# game_name = 'cyberpunk2077_phantom_liberty'

# game_steamid = 1091500          # cyberpunk2077
# game_name = 'cyberpunk2077'

# game_steamid = 1118010
# game_name = 'monster_hunter_world_iceborne'

# game_steamid = 582010
# game_name = 'monster_hunter_world'

game_steamid = 730
game_name = 'counter-strike_2'

params = {
    'json':1,
    'cursor':'*',
    'num_per_page':100,
    'language':'english',       # only english reviews
    'filter':'recent',
    'review_type':'all',
    'purchase_type':'all'

}

# api doc: https://partner.steamgames.com/doc/store/getreviews
# reviews_req = requests.get(f"https://store.steampowered.com/appreviews/{game_steamid}", params=params)

In [3]:
# loop to get comments in range of a day

datetime_until = datetime(2024, 1, 1, 0, 0, 0)      # get until...

checkpoint_counter = 0
reviews_reqs = []

# get existing folder and retrieve the cursor object (?)

# load the latest file
game_folder = Path(f'./{game_name}')
if game_folder.exists():
    try:
        latest_file_path = sorted(game_folder.glob('steam_reviews_*.pkl'))[-1]
        with open(latest_file_path, 'rb') as f:
            reviews_reqs = pickle.load(f)           # retrieve the list of reviews
            cursor_str = reviews_reqs[-1]['cursor']
            print('Loaded:', latest_file_path)
    except IndexError as e:
        print('Error loading the latest file:', e)
        traceback.print_exc()
else:
    cursor_str = '*'

print('Cursor:', cursor_str)


save_folder = game_folder
if not save_folder.exists():
    os.makedirs(save_folder)

while True:
    params['cursor'] = cursor_str
    reviews_req = requests.get(f"https://store.steampowered.com/appreviews/{game_steamid}", params=params)

    reviews_json = reviews_req.json()
    if 'reviews' not in reviews_json:
        print('No reviews found. To the end of all reviews')
        break
    reviews = reviews_json['reviews']
    if len(reviews) == 0:
        print('No reviews found. To the end of all reviews')
        break
    reviews_reqs.append(reviews_json)
    cursor_str = reviews_json['cursor']
    print(f"Got {len(reviews)} reviews, cursor: {cursor_str}")
    
    if datetime.fromtimestamp(reviews[-1]['timestamp_created']) < datetime_until:
        print('Reached until date')
        break

    checkpoint_counter += 1
    if checkpoint_counter % 50 == 0:
        save_filepath = save_folder.joinpath(f'steam_reviews_{game_steamid}_{datetime.now().strftime("%Y%m%d%H%M%S")}.pkl')
        with open(save_filepath, 'wb') as f:
            pickle.dump(reviews_reqs, f)
        print('Saved to:', save_filepath)
        # time.sleep(5)

    time.sleep(1)

# save the reviews
with open(save_folder.joinpath(f'steam_reviews_{game_steamid}_{datetime.now().strftime("%Y%m%d%H%M%S")}.pkl'), 'wb') as f:
    pickle.dump(reviews_reqs, f)
print('Program terminated. Saved to:', save_folder.joinpath(f'steam_reviews_{game_steamid}_{datetime.now().strftime("%Y%m%d%H%M%S")}.pkl'))

Cursor: *
Got 100 reviews, cursor: AoJwv5ad8o0DcY/c3wQ=
Got 100 reviews, cursor: AoJ49cKu8Y0Dff3R3wQ=
Got 100 reviews, cursor: AoJ4toL38I0DfMLK3wQ=
Got 100 reviews, cursor: AoJ4hPzG8I0Dca3C3wQ=
Got 100 reviews, cursor: AoJwsoCQ8I0Ddfq43wQ=
Got 100 reviews, cursor: AoJwyMu9740DfOmu3wQ=
Got 100 reviews, cursor: AoJwgNnt7o0DftSl3wQ=
Got 100 reviews, cursor: AoJwttHA7o0Dd8yf3wQ=
Got 100 reviews, cursor: AoJwp+aR7o0DfI2Y3wQ=
Got 100 reviews, cursor: AoJ467nM7Y0DdJGM3wQ=
Got 100 reviews, cursor: AoJwobz57I0DdsGB3wQ=
Got 100 reviews, cursor: AoJ4ncip7I0Dd5D43gQ=
Got 100 reviews, cursor: AoJ4g7T7640DeZ7y3gQ=
Got 100 reviews, cursor: AoJwysfY640DcInt3gQ=
Got 100 reviews, cursor: AoJ4rIuV640Ddenh3gQ=
Got 100 reviews, cursor: AoJw4vLK6o0DcsnY3gQ=
Got 100 reviews, cursor: AoJ46Kje6Y0DdoDN3gQ=
Got 100 reviews, cursor: AoJwxcqm6Y0DfZDG3gQ=
Got 100 reviews, cursor: AoJw75OK6Y0DdYrC3gQ=
Got 100 reviews, cursor: AoJwnOvo6I0DdeK83gQ=
Got 100 reviews, cursor: AoJw6Ouo6I0Dcb+z3gQ=
Got 100 reviews, cursor:

KeyboardInterrupt: 

In [4]:
len(reviews_reqs)

2551

In [5]:
datetime.fromtimestamp(1707534074)      # its a unix timestamp, to our timezone (GMT+8).

datetime.datetime(2024, 2, 10, 11, 1, 14)

In [5]:
reviews_req.json()

{'success': 1,
 'query_summary': {'num_reviews': 100},
 'reviews': [{'recommendationid': '135572389',
   'author': {'steamid': '76561199131006909',
    'num_games_owned': 0,
    'num_reviews': 1,
    'playtime_forever': 20317,
    'playtime_last_two_weeks': 0,
    'playtime_at_review': 19358,
    'last_played': 1706881986},
   'language': 'english',
   'review': 'iyttg',
   'timestamp_created': 1679956949,
   'timestamp_updated': 1679956949,
   'voted_up': True,
   'votes_up': 1,
   'votes_funny': 0,
   'weighted_vote_score': '0.523809552192687988',
   'comment_count': 0,
   'steam_purchase': True,
   'received_for_free': False,
   'written_during_early_access': False,
   'hidden_in_steam_china': True,
   'steam_china_location': ''},
  {'recommendationid': '135572318',
   'author': {'steamid': '76561199455439899',
    'num_games_owned': 0,
    'num_reviews': 1,
    'playtime_forever': 3629,
    'playtime_last_two_weeks': 0,
    'playtime_at_review': 1313,
    'last_played': 1705121744}

In [6]:
with open('reviews.json', 'w') as f:
    json.dump(reviews_req.json(), f, indent=2)

In [4]:
print(reviews_req.json()['reviews'][2]['review'])

Was initially disappointed with this game but its come along way and I seriously enjoy it. Cant wait for CK to be added since modding it right now is a massive pain
